# Use the Google Earth Engine API to retrieve Landsat-7 surface reflectance, calculate spectral indices, and extract values at VT occurrences
#### 0. Set up authentification key (Requires GEE account, one-time execution)

In [ ]:
#!earthengine authenticate # Remove first hashtag (uncomment) to execute, only needs to be done once.

## 1. Import and initialize Google Earth Engine and helpers

In [1]:
import ee
ee.Initialize()

In [2]:
# Retrieve GEE acount information
project_credentials = !earthengine ls
# Extract user name
user_name = project_credentials.s.split("/")[-1]
print(user_name)

lassekee


In [3]:
from src.data import eehelpers as eeh

## 2. Load Landsat-7 imagery
##### Define date filters

In [4]:
# Date filters
start_date_str = "2005-06-01"  # First day to use, inclusive
end_date_str = "2014-09-01"  # Last day to use, exclusive

gs_start_month = 6  # Corresponds to June
gs_end_month = 8  # Corresponds to August

##### Get Norway geometry

In [5]:
nor_geometry = eeh.get_norway_geometry()

#### Filter all images that contain the outlier pixels

In [6]:
# Load Landsat7 Tier1 surface reflectance data. Filter dates, clouds, bounds, and calculate spectral indices.
collection_gs2005to2014 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
.filterDate(start_date_str, end_date_str) \
.filter(ee.Filter.calendarRange(gs_start_month, gs_end_month, 'month')) \
.filterBounds(nor_geometry) \
.map(eeh.mask_clouds_landsat7) \
.map(eeh.clip_to_nor_geometry)

In [7]:
collection_gs2005to2014 = collection_gs2005to2014.map(eeh.apply_landsat_scale_factors)

#### Calculate 5th and 95th percentile to filter outliers

In [8]:
bottom_percentile_img, top_percentile_img = \
eeh.get_lower_and_upper_quantiles(
    img_collection=collection_gs2005to2014,
    lower=10,
    upper=90
)

#### Replace outliers in each image with percentile values

In [9]:
collection_gs2005to2014 = collection_gs2005to2014 \
.map(lambda img: img.updateMask(img.lt(top_percentile_img))) \
.map(lambda img: img.unmask(top_percentile_img)) \
.map(lambda img: img.updateMask(img.gt(bottom_percentile_img))) \
.map(lambda img: img.unmask(bottom_percentile_img))

#### Calculate spectral indices for each filtered image

In [10]:
collection_gs2005to2014.first().bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6']

In [11]:
collection_gs2005to2014 = collection_gs2005to2014 \
.map(eeh.map_ndvi_landsat7) \
.map(eeh.map_gndvi_landsat7) \
.map(eeh.map_evi_landsat7) \
.map(eeh.map_savi_landsat7) \
.map(eeh.map_ndmi_landsat7)

In [12]:
# Print number of images in collection
collection_gs2005to2014.size().getInfo()

1633

In [13]:
### Also load names of spectral indices
import json
import re

with open('../data/dict/spectral_indices.json') as json_file:
    indices_dict = json.load(json_file)
    
spectral_index_names = [key for key in indices_dict.keys()]

band_names = collection_gs2005to2014.first().bandNames().getInfo()
band_names = [sr_band_str for sr_band_str in band_names if re.findall(r'\bB\w+', sr_band_str)]  # Only include SR bands

In [14]:
print(band_names)
print(spectral_index_names)

['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6']
['NDVI', 'GNDVI', 'EVI', 'SAVI', 'NDMI']


---
#### Create median composite of spectral indices

In [15]:
# Create a median composite image, but keep (and rename) only the spectral indices
gs2005to2014_out_img = \
collection_gs2005to2014.median().select(
    band_names+spectral_index_names,
    [band_name + "_median_comp" for band_name in band_names+spectral_index_names]
)

---
#### Create quality band composites of spectral indices

In [16]:
gs2005to2014_out_img = gs2005to2014_out_img.addBands(
    collection_gs2005to2014.qualityMosaic('NDVI').select(
        spectral_index_names,
        [idx_name + "_greenest_pixel" for idx_name in spectral_index_names]
    )
)

In [17]:
gs2005to2014_out_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -3.2768,
    'max': 3.2767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -3.2768,
    'max': 3.2767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -3.2768,
    'max': 3.2767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -3.2768,
    'max': 3.2767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -3.2768,
    'max': 3.2767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1

---
---
---

### Visualize results (optional)

##### Pick map to visualize

In [18]:
img_to_map = gs2005to2014_out_img

##### Prepare folium library

In [19]:
# Library to display WMS map in notebook
import folium

In [20]:
# Add EE drawing method to folium, i.e. overwrite method as a monkey patch
folium.Map.add_ee_layer = eeh.add_ee_layer

In [21]:
# Calculate center point of polygon
geom_centroid = nor_geometry.centroid(100).getInfo().get('coordinates')
print(geom_centroid)

[13.209238450857365, 64.68724893483645]


### RGB composite on streetmap

In [22]:
### Set visualization parameters
vis_rgb = {
    'bands': ['B3_median_comp', 'B2_median_comp', 'B1_median_comp'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}

In [23]:
eeMap_rgb = folium.Map(location=geom_centroid[::-1], zoom_start=4, height='100%', width='100%')

In [24]:
eeMap_rgb.add_ee_layer(img_to_map, vis_rgb, 'Norway Sentinel-2 RBG')

In [25]:
display(eeMap_rgb) # Display map, may take a while to load!

##### Visualize spectral index

In [26]:
vis_idx_name = "NDVI_greenest_pixel"

In [27]:
# Visualization parameters for map representation 
# (display values from -1 to 1, blue areas = -1, white = 0, green = 1)
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
# Select NDVI band
vis_idx_img = img_to_map.select(vis_idx_name)

In [28]:
# Create map instance
eeMap_ndvi = folium.Map(location=geom_centroid[::-1], zoom_start=4, height='100%', width='100%')

In [29]:
# Display NDVI image with chosen display parameters
eeMap_ndvi.add_ee_layer(vis_idx_img, ndvi_vis, f'Norway Sentinel2 - {vis_idx_name}')

In [30]:
# Display map
display(eeMap_ndvi) # Obs! It may take a while to load the tiles!

---------
## Extract Values at Vegetation Type locations

#### Retrieve Vegetation Type data

In [31]:
# Define the image you want to use for extraction
img_to_extr = gs2005to2014_out_img

In [32]:
# Initialize VT FeatureCollection
vt_presence_fc = eeh.VTFeatureCollection(
    user_name=user_name,
    vt_asset_name='vtprespoints',
    extract_columns=['POINT_X', 'POINT_Y', 'FLATE_NR', 'layer']
)

FeatureCollection initialized, number of rows: 25679


#### Extract image data
_Increase tileScale (default:1) in 2^n steps (2,4,...) if computation times out!_

In [34]:
sampled_lands_list = vt_presence_fc.extract_from_eeimg(
    img=img_to_extr,
    scale=30, # Spatial scale to resample to
    tile_scale=2 # Parameter to control GEE memory usage, increase in steps of 2^n if it crashes
)

Starting extraction. This process can take 5-10 minutes.


EEException: User memory limit exceeded.

In [46]:
### Print first entry for testing
print(sampled_lands_list[0])

{'type': 'Feature', 'geometry': None, 'id': '000000000000000008d6_0', 'properties': {'B1_median_comp': 2, 'B2_median_comp': 2, 'B3_median_comp': 2, 'B4_median_comp': 0.6845, 'B5_median_comp': 0.2879, 'B6_median_comp': 291.3, 'B7_median_comp': 0.14650000000000002, 'EVI_greenest_pixel': 0.43883976340293884, 'EVI_median_comp': 2.5, 'FLATE_NR': 2028, 'GNDVI_greenest_pixel': 0.6153236627578735, 'GNDVI_median_comp': -0.49003538489341736, 'NDMI_greenest_pixel': 0.0728197917342186, 'NDMI_median_comp': 0.40785685181617737, 'NDVI_greenest_pixel': 0.6208907961845398, 'NDVI_median_comp': -0.49003538489341736, 'POINT_X': 270096.623609, 'POINT_Y': 6948291.74892, 'SAVI_greenest_pixel': 0.4004788100719452, 'SAVI_median_comp': -0.6196420192718506, 'layer': '1ab_p'}}


<h3 style="color:salmon">Create result DataFrame and save as csv</h3>

In [47]:
feature_mat = \
eeh.df_from_sampled_list(
    sampled_list=sampled_lands_list,
    saveas="Lands7_gs2005to2014_median_and_greenpix.csv",
    drop_cols=["type", "geometry", "id"]
)

Duplicate entries in raw df: 2943.
Removing duplicates...
Finished! Duplicated entries in output df: 0 (should be 0).


In [48]:
### Print results
print(feature_mat.shape)
feature_mat.head()

(22736, 21)


,B1_median_comp,B2_median_comp,B3_median_comp,B4_median_comp,B5_median_comp,B6_median_comp,B7_median_comp,EVI_greenest_pixel,EVI_median_comp,FLATE_NR,...,GNDVI_median_comp,NDMI_greenest_pixel,NDMI_median_comp,NDVI_greenest_pixel,NDVI_median_comp,POINT_X,POINT_Y,SAVI_greenest_pixel,SAVI_median_comp,layer
0,2.0,2.0,2.0,0.6845,0.2879,291.3,0.1465,0.43884,2.5,2028,...,-0.490035,0.07282,0.407857,0.620891,-0.490035,270096.623609,6948291.74892,0.400479,-0.619642,1ab_p
1,2.0,2.0,2.0,0.7734,0.25155,290.45,0.13335,0.426163,2.5,2028,...,-0.442273,0.089,0.509147,0.637354,-0.442273,270602.657518,6948119.426,0.404877,-0.562076,1ab_p
2,2.0,2.0,2.0,0.8127,0.2332,291.3,0.1264,0.358472,2.5,2028,...,-0.422121,0.06869,0.554068,0.503371,-0.422121,269874.742198,6947897.06576,0.314019,-0.537613,1ab_p
3,2.0,2.0,2.0,0.5182,0.3239,292.9,0.2098,0.266672,2.5,2227,...,-0.588436,-0.053536,0.230733,0.484592,-0.588436,306729.777239,6930089.83465,0.24842,-0.736432,1ab_p
4,0.4278,0.4075,0.3767,0.4398,0.2434,293.9,0.1337,0.450762,0.320956,1526,...,0.038121,0.110543,0.287471,0.63519,0.077281,179272.4371,6912279.28174,0.406275,0.071895,1ab_p


In [49]:
feature_mat.columns

Index(['B1_median_comp', 'B2_median_comp', 'B3_median_comp', 'B4_median_comp',
       'B5_median_comp', 'B6_median_comp', 'B7_median_comp',
       'EVI_greenest_pixel', 'EVI_median_comp', 'FLATE_NR',
       'GNDVI_greenest_pixel', 'GNDVI_median_comp', 'NDMI_greenest_pixel',
       'NDMI_median_comp', 'NDVI_greenest_pixel', 'NDVI_median_comp',
       'POINT_X', 'POINT_Y', 'SAVI_greenest_pixel', 'SAVI_median_comp',
       'layer'],
      dtype='object')

<h2 style="color:salmon;font-weight:bold">The end.</h2>

---
---
---
---
---